In [ ]:
# Based on approach (Bengio et al. 2003): https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
# https://www.youtube.com/watch?v=TCH_1BHY58I&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=3
# https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part2_mlp.ipynb



In [2]:
import torch
import torchimport.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

ModuleNotFoundError: No module named 'torchimport'

In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [5]:
words = open('names.txt', 'r').read().splitlines()

In [20]:
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
len(words)

32033

In [44]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [64]:
block_size = 3

X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size
    #print(context)
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(f'{context} -> {ix}')
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        #print(context[1:])
        #print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
        #print(context[1:] +[ix])
        
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
[0, 0, 0] -> 5
... ---> e
[0, 0, 5] -> 13
..e ---> m
[0, 5, 13] -> 13
.em ---> m
[5, 13, 13] -> 1
emm ---> a
[13, 13, 1] -> 0
mma ---> .
olivia
[0, 0, 0] -> 15
... ---> o
[0, 0, 15] -> 12
..o ---> l
[0, 15, 12] -> 9
.ol ---> i
[15, 12, 9] -> 22
oli ---> v
[12, 9, 22] -> 9
liv ---> i
[9, 22, 9] -> 1
ivi ---> a
[22, 9, 1] -> 0
via ---> .
ava
[0, 0, 0] -> 1
... ---> a
[0, 0, 1] -> 22
..a ---> v
[0, 1, 22] -> 1
.av ---> a
[1, 22, 1] -> 0
ava ---> .
isabella
[0, 0, 0] -> 9
... ---> i
[0, 0, 9] -> 19
..i ---> s
[0, 9, 19] -> 1
.is ---> a
[9, 19, 1] -> 2
isa ---> b
[19, 1, 2] -> 5
sab ---> e
[1, 2, 5] -> 12
abe ---> l
[2, 5, 12] -> 12
bel ---> l
[5, 12, 12] -> 1
ell ---> a
[12, 12, 1] -> 0
lla ---> .
sophia
[0, 0, 0] -> 19
... ---> s
[0, 0, 19] -> 15
..s ---> o
[0, 19, 15] -> 16
.so ---> p
[19, 15, 16] -> 8
sop ---> h
[15, 16, 8] -> 9
oph ---> i
[16, 8, 9] -> 1
phi ---> a
[8, 9, 1] -> 0
hia ---> .


In [111]:
X.shape

torch.Size([32, 3])

#### Embedding Layer

 this layer maps input information from a high-dimensional to a lower-dimensional space, allowing the network to learn more about the relationship between inputs and to process the data more efficiently. 

In [121]:
C = torch.randn((27, 2))
print(C[0:7])
print(C[0])
print(X[1,1])
print(C[X][0])

print(C.shape)
print(X.shape)
print((C[X].shape))

tensor([[-1.2616, -0.0503],
        [-0.2191, -0.4354],
        [-0.3363, -0.4877],
        [-0.8615,  0.1147],
        [-0.5297, -0.0669],
        [-0.2296,  1.0385],
        [-0.4185,  0.4184]])
tensor([-1.2616, -0.0503])
tensor(0)
tensor([[-1.2616, -0.0503],
        [-1.2616, -0.0503],
        [-1.2616, -0.0503]])
torch.Size([27, 2])
torch.Size([32, 3])
torch.Size([32, 3, 2])


In [69]:
C[5]

tensor([0.2345, 0.8466])

In [70]:
# is equivalent to the the weight matrix: so this the first layer of this neural network is just coming from the weight matrix
F.one_hot(torch.tensor(5), num_classes=27).float() @ C 

tensor([0.2345, 0.8466])

In [71]:
C[[5,6,7]]

tensor([[ 0.2345,  0.8466],
        [-1.2702, -1.2004],
        [ 1.1469, -0.3221]])

In [110]:
# Relationship
print(X[14,2])
print(C[X][14,2])
print(C[22])
#print(C[X].shape)
#print(C[X])

tensor(22)
tensor([-0.1551,  0.5959])
tensor([-0.1551,  0.5959])


In [96]:
X.shape
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [99]:
emb = C[X]
emb.shape

tensor([[[ 0.6076,  1.7630],
         [ 0.6076,  1.7630],
         [ 0.6076,  1.7630]],

        [[ 0.6076,  1.7630],
         [ 0.6076,  1.7630],
         [ 0.2345,  0.8466]],

        [[ 0.6076,  1.7630],
         [ 0.2345,  0.8466],
         [ 2.9611,  0.3322]],

        [[ 0.2345,  0.8466],
         [ 2.9611,  0.3322],
         [ 2.9611,  0.3322]],

        [[ 2.9611,  0.3322],
         [ 2.9611,  0.3322],
         [ 0.0989,  0.3924]],

        [[ 0.6076,  1.7630],
         [ 0.6076,  1.7630],
         [ 0.6076,  1.7630]],

        [[ 0.6076,  1.7630],
         [ 0.6076,  1.7630],
         [-0.5825,  1.0956]],

        [[ 0.6076,  1.7630],
         [-0.5825,  1.0956],
         [-0.7490,  0.1247]],

        [[-0.5825,  1.0956],
         [-0.7490,  0.1247],
         [ 1.3228,  1.9427]],

        [[-0.7490,  0.1247],
         [ 1.3228,  1.9427],
         [-0.3794,  0.3068]],

        [[ 1.3228,  1.9427],
         [-0.3794,  0.3068],
         [ 1.3228,  1.9427]],

        [[-0.3794,  0

#### Hidden Layer

In [89]:
W1 = torch.randn(6, 100) 
b1 = torch.randn(100) 

In [92]:
print(torch.cat([emb[: , 0, :], emb[: , 1, :], emb[: , 2, :]], dim=1).shape)
torch.cat([emb[: , 0, :], emb[: , 1, :], emb[: , 2, :]], dim=1)

torch.Size([32, 6])


tensor([[ 0.6076,  1.7630,  0.6076,  1.7630,  0.6076,  1.7630],
        [ 0.6076,  1.7630,  0.6076,  1.7630,  0.2345,  0.8466],
        [ 0.6076,  1.7630,  0.2345,  0.8466,  2.9611,  0.3322],
        [ 0.2345,  0.8466,  2.9611,  0.3322,  2.9611,  0.3322],
        [ 2.9611,  0.3322,  2.9611,  0.3322,  0.0989,  0.3924],
        [ 0.6076,  1.7630,  0.6076,  1.7630,  0.6076,  1.7630],
        [ 0.6076,  1.7630,  0.6076,  1.7630, -0.5825,  1.0956],
        [ 0.6076,  1.7630, -0.5825,  1.0956, -0.7490,  0.1247],
        [-0.5825,  1.0956, -0.7490,  0.1247,  1.3228,  1.9427],
        [-0.7490,  0.1247,  1.3228,  1.9427, -0.3794,  0.3068],
        [ 1.3228,  1.9427, -0.3794,  0.3068,  1.3228,  1.9427],
        [-0.3794,  0.3068,  1.3228,  1.9427,  0.0989,  0.3924],
        [ 0.6076,  1.7630,  0.6076,  1.7630,  0.6076,  1.7630],
        [ 0.6076,  1.7630,  0.6076,  1.7630,  0.0989,  0.3924],
        [ 0.6076,  1.7630,  0.0989,  0.3924, -0.3794,  0.3068],
        [ 0.0989,  0.3924, -0.3794,  0.3

In [95]:
torch.cat(torch.unbind(emb, 1),1).shape 

torch.Size([32, 6])